In [ ]:
import PIL
from PIL import Image
import keras
import glob2
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Rescaling, RandomTranslation, RandomRotation, RandomZoom, RandomContrast, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2, ResNet50V2, ResNet152V2, VGG16, EfficientNetB0
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tabulate
import random
from matplotlib.image import imread

In [ ]:
#Image Processing

In [ ]:
#import training images
imgs = []
for i in range(0,43):
    i = str(i)
    for filename in glob2.glob('C:/Users/Nathan/Documents/STA 208/Final Project/Train/' + i + '/*',recursive  = True):
        imgs = np.append(imgs,filename)

#opens each image in training set, adjusts string naming, changes images to grayscale and save as image
count = 0
for i in imgs:
    count = str(count)
    #open image
    img = Image.open(i)
    #convert string naming
    dummy = train_box.loc[lambda train_box: train_box['Path'] == trainimgs_names[int(count)]]
    dum2 = dummy.to_numpy()
    dum3 = str(dum2).replace('[[','').replace(']]','').split()
    dum4 = np.array(dum3)[:-1].astype(int)
    ID = dum4[6] 
    #convert image to grayscale and resave file in directory
    img.convert("L").save('Grayscale/'+str(ID)+'/' + count + '.png')
    count = int(count) +1
    
#import test images
testimgs = []
for filename in glob2.glob('C:/Users/Nathan/Documents/STA 208/Final Project/Test/*',recursive  = True):
    testimgs.append(filename)

#opens images, converts to grayscale and resaves as number
count = 0
for i in testimgs:
    count = str(count)
    img = Image.open(i)
    img.convert("L").save('Grayscale_Test/' + count + '.png')
    count = int(count)+1
  
#import training and testing metafiles, contains information on how to crop each image to only include traffic sign
train_box = pd.read_csv("Train.csv")
test_box = pd.read_csv("Test.csv")

#import test images file paths
testimgs_names = []
for i in testimgs:
    testimgs_names.append(i.replace('C:/Users/Nathan/Documents/STA 208/Final Project/',''))
#import training images file paths
trainimgs_names = []
for i in imgs:
    trainimgs_names.append(i.replace('C:/Users/Nathan/Documents/STA 208/Final Project/',''))

#take each test images, converts string naming, uses metafiles to crop image by given information
for i in testimgs:
    count = str(count)
    #import image
    img = Image.open(i)
    #rename string, needed for filepath
    dummy = test_box.loc[lambda test_box: test_box['Path'] == testimgs_names[int(count)]]
    dum2 = dummy.to_numpy()
    dum3 = str(dum2).replace('[[','').replace(']]','').split()
    dum4 = np.array(dum3)[:-1].astype(int)
    #get crop distances from metafile
    x1 = dum4[2]
    y1 = dum4[3]
    x2 = dum4[4]
    y2 = dum4[5]
    #crop and resave image
    img.crop(box=(x1,y1,x2,y2)).save('Cropped_Test/'+count+'.png')
    count = int(count) +1

count = 0

#convert training images to cropped versions given metafile information
for i in imgs:
    count = str(count)
    #open image
    img = Image.open(i)
    #find given image crop parameters
    dummy = train_box.loc[lambda train_box: train_box['Path'] == trainimgs_names[int(count)]]
    dum2 = dummy.to_numpy()
    dum3 = str(dum2).replace('[[','').replace(']]','').split()
    dum4 = np.array(dum3)[:-1].astype(int)
    #save crop parameters
    x1 = dum4[2]
    y1 = dum4[3]
    x2 = dum4[4]
    y2 = dum4[5]
    ID = dum4[6] 
    #crop image and resave
    img.crop(box=(x1,y1,x2,y2)).save('Cropped/'+str(ID)+'/'+count+'.png')
    count = int(count) +1

In [ ]:
#EDA Plots

In [ ]:
#import metafiles for EDA plot
traindf = pd.read_csv("Train.csv")
testdf = pd.read_csv("Test.csv")

#combine training and testing into total data file
combdf = pd.concat([traindf,testdf],axis=0)

#adjust figure size
sns.set(rc={'figure.figsize':(12,8)})
#fix background
sns.set_style("darkgrid")
#fix font size
sns.set_context('talk')
#plot histogram of pixel heights and widths for all images
sns.histplot(combdf, x="Width",y="Height",bins=10,cbar=True).set_title("Image Pixel Dimensions: Width vs Height")
#save histogram for report
#plt.savefig("Width_by_Height_Hist.png")

#creat histogram of traffic sign class image totals, not for plot but for information
density, bins, _ = plt.hist(data=combdf,x='ClassId',bins=43)
count, _ = np.histogram(combdf['ClassId'], bins)

#adjust figure size
sns.set(rc={'figure.figsize':(10,15)})
#adjust text size and background
sns.set_context('talk')
sns.set_style("darkgrid")
#plot count plot for image totals for each traffic sign class
plot = sns.countplot(data=combdf, y="ClassId",palette=["teal",'gold'])
#set labels
plot.set_ylabel("Traffic Sign Class ID")
plot.set_xlabel("Total # of Images")
plot.set_title("GTSR Dataset: Traffic Sign Class Totals")
#sets x range
plt.xlim(0,3300)

#plots bin totals on plot
bins = range(0,43)
for x,y,num in zip(bins, density, count):
    if num != 0:
        plt.text(y, x+.4, num, fontsize=18)

#save image for report
#plt.savefig("Class_Image_Counts_plot.png")

#adjust figure size, text size, and background
sns.set(rc={'figure.figsize':(15,10)})
sns.set_context('talk')
sns.set_style("darkgrid")
#plot countplot for each traffic sign class ID (opposite axis)
plot = sns.countplot(data=combdf, x="ClassId",palette=["teal",'gold'])
#set label names
plot.set_xlabel("Traffic Sign Class ID")
plot.set_ylabel("Total # of Images")
plot.set_title("GTSR Dataset: Traffic Sign Class Totals")
#set y range
plt.ylim(0,3300)

bins = range(0,43)

#add bin text to plot
for x,y,num in zip(bins, density, count):
    if num != 0:
        plt.text(x-.4, y+50, num, fontsize=18,rotation=90)
        
#save plot for report
#plt.savefig("Class_Image_Counts_plot_horizontal.png")

#adjust figure size
sns.set(rc={'figure.figsize':(12,10)})

#read in chosen images for example of each class
paths = pd.read_excel("ImageGridPaths.xlsx")
#load filepaths
imgs = paths['Path'].values
data_dir = 'C:/Users/Nathan/Documents/STA 208/Final Project'

#se tplot size
plt.figure(figsize=(20,20))

#plot in a (9,5) grid of each traffic sign class
for i in range(1,44):
    #set up subplot
    plt.subplot(9,5,i)
    #remove ticks from subplot for images
    plt.setp(plt.subplot(9,5,i).get_xticklabels(), visible=False)
    plt.setp(plt.subplot(9,5,i).get_yticklabels(), visible=False)
    #title each subplot with class ID
    plt.subplot(9,5,i).title.set_text('Class ID: ' + str(i-1))
    #take file path from images directory
    random_img_path = data_dir + '/' + imgs[i-1]
    #load image
    rand_img = imread(random_img_path)
    #plot image
    plt.imshow(rand_img)
    #remove grid in image to just show image
    plt.grid(visible=None)
    #show pixel width and height next to plot
    #plt.xlabel(rand_img.shape[1], fontsize = 20)#width of image
    #plt.ylabel(rand_img.shape[0], fontsize = 20)#height of image
    
#save plot grid of traffic sign classes for report
#plt.savefig("Traffic_Sign_Image_Grid.png")

plt.figure(figsize=(20,20))
title = ['Original','Cropped','Grayscale']
paths = ['C:/Users/Nathan/Documents/STA 208/Final Project/Train/0/00000_00000_00029.png',
         'C:/Users/Nathan/Documents/STA 208/Final Project/Cropped/0/29.png',
         'C:/Users/Nathan/Documents/STA 208/Final Project/Grayscale/0/29.png']

for i in range(1,4):
    plt.subplot(1,3,i)
    plt.setp(plt.subplot(1,3,i).get_xticklabels(), visible=False)
    plt.setp(plt.subplot(1,3,i).get_yticklabels(), visible=False)
    plt.subplot(1,3,i).title.set_text(title[i-1])
    random_img_path = paths[i-1]
    rand_img = imread(random_img_path)
    plt.imshow(rand_img)
    #plt.grid(False)
    plt.grid(visible=None)
#plt.savefig("Original_Cropped_Grayscale_Image.png")